В один прекрасный день мне понадобилась база данных с анекдотами. Если быть точным, я решил добавить в своего голосового ассистента функцию, которая по запросу "Расскажи анекдот" выдавала бы случайный анекдот. Пользоваться данной функцией приходилось в местах, где нет интернета, поэтому было решено спарсить базу анекдотов, желательно большого размера.

Я решил парсить сайт http://anekdotme.ru/random на странице которого при каждом обращении случайным образом показывались 30 анекдотов.

Недолго думая я набросал скрипт на Python который парсит базу и получает на выходе .db файл содержащий 90 тысяч анекдотов. Время парсинга такого огромного количества анекдотов - примерно 3-4 часа, причем желательно не выключать скрипт, не дождавшись пока он не отработает до конца.


Оригинал текста взят с сайта https://pythono.ru/parsing-python-bs4/

In [1]:
import requests, bs4
import sqlite3
import re
import time

from random import randint
from time import sleep

connection = sqlite3.connect('anekdot.db')
cursor = connection.cursor()
cursor.executescript("""create table anekdot(
        id int auto_increment primary key, anekdot longtext );""")
z=0
for i in range(4000):
    sleep(randint(1,10))
    z=z+1
    s=requests.get('http://anekdotme.ru/random')
    b=bs4.BeautifulSoup(s.text, "html.parser")
    p=b.select('.anekdot_text')
    for x in p:        
        s=(x.getText().strip())
        reg = re.compile('[^a-zA-Zа-яА-я .,!]')
        s=reg.sub('', s)
        cursor.execute("INSERT INTO anekdot (anekdot) VALUES ('"+s+"')")
        connection.commit()
    print(z)
connection.close()

1
2
3
4
5
6
7
8


ConnectionError: HTTPConnectionPool(host='anekdotme.ru', port=80): Max retries exceeded with url: /random (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x000000000815A470>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))

При каждом запуске скрипт покажет нам случайный анекдот из спарсенной базы. Что делать с такой базой? Да что угодно - так как формат SQLi довольно хорошо поддерживается во всех языках программирования, можно создать себе на сайт виджет случайного анекдота с помощью PHP, запилить оффлайн-сборник анекдотов для Android или iOS. 


In [12]:
import sqlite3
import random
connection = sqlite3.connect('anekdot.db')
cursor = connection.cursor()
z=random.randrange(1,100,1)
cursor.execute('SELECT * FROM anekdot WHERE rowid='+str(z))
row = cursor.fetchone()
print(str(row[1]))
connection.close()

Экзамен на режиссерском факе ВГИКа. Студентов спрашивают   Допустим, кадр мужчина и женщина пришли вместе в гости. Как сразу дать понять зрителям, что это муж и жена  Один студент предлагает   Она поправляет ему галстук.   А может, это его секретарша  Другой   Он нежно целует ее в прихожей.   Ну, это скорее любовники.  Третий   Он ворчит зачем я на тебе женился....   А может, они уже в разводе Четвертый   Он достает у нее из сумочки платок, сморкается в него и сует обратно в сумочку.   Пять баллов!
